<a href="https://colab.research.google.com/github/SamSzamocki/SamSzamocki_playground/blob/main/Ch5_2_Slice_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"


--2024-07-23 14:54:50--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [       <=>          ]  41.00M  27.8MB/s    in 1.5s    

2024-07-23 14:54:51 (27.8 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]



In [2]:
!unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
!pip install --upgrade pyarrow datasets

from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}

drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))

drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [5]:
import pandas as pd

df = pd.DataFrame(drug_sample)

df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,87571,Naproxen,"Gout, Acute","""like the previous person mention, I&#039;m a ...",9.0,"September 2, 2015",36
1,178045,Duloxetine,ibromyalgia,"""I have taken Cymbalta for about a year and a ...",3.0,"November 7, 2011",13
2,80482,Mobic,Inflammatory Conditions,"""I have been taking Mobic for over a year with...",10.0,"June 5, 2013",128
3,159268,TriNessa,Birth Control,"""I am now 21 and have been on TriNessa since I...",10.0,"October 21, 2010",81
4,205477,Pristiq,Depression,"""I was originally on Prozac 20mg for my depres...",10.0,"July 13, 2013",65


In [6]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [7]:
rows_train = len(drug_dataset["train"])
rows_test = len(drug_dataset["test"])
rows = len(drug_dataset["train"]) + len(drug_dataset["test"])

rows_unique_id = len(drug_dataset["train"].unique("Unnamed: 0"))

print(rows_unique_id)
print(rows_train)
print(rows_test)
print(rows)

161297
161297
53766
215063


In [8]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="id"
)

drug_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [9]:
unique_drugs = set()

unique_conditions = set()

for split in drug_dataset.keys():
    df = drug_dataset[split].to_pandas()
    unique_drugs.update(df["drugName"].unique())
    unique_conditions.update(df["condition"].unique())

print(len(unique_drugs))
print(len(unique_conditions))

3671
917


In [10]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

#this lowercases all the text fields in the condition column,
#however there are some None objects which need to be filtered out first


In [11]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [12]:
#now we can apply the lower case method using map
drug_dataset = drug_dataset.map(lowercase_condition)



Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [13]:
print(drug_dataset["train"]["condition"][:3])

['left ventricular dysfunction', 'adhd', 'birth control']


In [14]:
train_df = drug_dataset["train"].to_pandas()

print("\nFirst 5 rows of the 'train' dataset:")
train_df.head()



First 5 rows of the 'train' dataset:


,id,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [15]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}


In [16]:
drug_dataset = drug_dataset.map(compute_review_length)

drug_dataset["train"][0]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [17]:
drug_dataset["train"].sort("review_length")[:3]




{'id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [18]:
#as the code contains some reviews with only 1 line, we need to remove the shorter ones through filtering

drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)

print(drug_dataset.num_rows)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}


In [19]:
#we need to remove all the html characters

import html

drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [20]:
#doing this batched allows us to compute in batches and much faster
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [21]:
#let's check how batching can affect tokenization speed
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["review"], padding=True, truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
#we can time how long it takes to run the cell using %time

%time new_drug_dataset = new_drug_dataset.map(tokenize, batched=True, batch_size=1000)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 2min 32s, sys: 3.66 s, total: 2min 36s
Wall time: 2min 2s


In [23]:
import torch

if torch.cuda.is_available():
    print("Running on GPU")
else:
    print("Running on CPU")


Running on CPU


In [24]:
#Let's use the tokenizer to split the data into 128 token sizes (i.e. truncate it) using the overflow arg

def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [25]:
#we test on one example first

result = tokenize_and_split(drug_dataset["train"][0])
[len(k) for k in result["input_ids"]]

[128, 45]

In [26]:
#now we apply the tokenize and split to the entire dataset however the below will results in an error

# tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [27]:
# The error indicates that since we used overflow_tokens, we've now got an input_ids column greater than the original column length of 1000.
# We can either make the old columns the same length or simply remove them with remove_columns
# As the tokenize_and_split function creates a new column, we can remove all the existing columns

tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [28]:
# Now we compare the old table to the new table
original_dataset_df = drug_dataset["train"].to_pandas()

original_dataset_df.head()

,id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68


In [29]:
tokenized_dataset_df = tokenized_dataset["train"].to_pandas()

tokenized_dataset_df.head()

,input_ids,attention_mask,overflow_to_sample_mapping
0,"[101, 1000, 2026, 2365, 2003, 8576, 2083, 2010...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1,"[101, 2625, 6832, 1006, 1037, 2204, 2518, 1007...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2,"[101, 1000, 1045, 2109, 2000, 2202, 2178, 8700...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[101, 3134, 1998, 2085, 2009, 1005, 1055, 1996...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[101, 1000, 2023, 2003, 2026, 2034, 2051, 2478...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


In [30]:
# We could also make it work by making the columns the same length

def tokenize_and_split(examples):
    # Tokenize with truncation and overflow
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
        padding="max_length"
    )

# extract mapping between old and new indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
      result[key] = [values[i] for i in sample_map]
    return result

In [31]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset_df = tokenized_dataset["train"].to_pandas()
tokenized_dataset_df.head()

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

,id,drugName,condition,review,rating,date,usefulCount,review_length,input_ids,attention_mask
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141,"[101, 1000, 2026, 2365, 2003, 8576, 2083, 2010...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141,"[101, 2625, 6832, 1006, 1037, 2204, 2518, 1007...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134,"[101, 1000, 1045, 2109, 2000, 2202, 2178, 8700...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134,"[101, 3134, 1998, 2085, 2009, 1005, 1055, 1996...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89,"[101, 1000, 2023, 2003, 2026, 2034, 2051, 2478...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [32]:
# We can use the set_format to set the format to pandas and view the data as a pandas dataframe rather than as a dictionary

drug_dataset.set_format("pandas")


In [33]:
drug_dataset["train"][0]

,id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141


In [34]:
# let's format the entire dataset to pandas and store it as a variable
train_df = drug_dataset["train"][:]

In [35]:
# Once the dataset is in pandas format, we can use pandas functions for data analysis

frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"count": "frequency"})
)
frequencies.head()

,condition,frequency
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


In [36]:
# now we can convery back into a usable dataset object using the dataset class
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['condition', 'frequency'],
    num_rows: 819
})

In [37]:
# let's compute the average rating per drug

average_rating = (
    train_df.groupby("drugName")["rating"]
    .mean()
    .to_frame()
    .reset_index()
    .rename(columns={"rating": "average_rating"})
)
average_rating.head()


,drugName,average_rating
0,A + D Cracked Skin Relief,10.000000
1,A / B Otic,10.000000
2,Abacavir / dolutegravir / lamivudine,7.953488
3,Abacavir / lamivudine / zidovudine,9.000000
4,Abatacept,7.312500


In [38]:
# let's reset the whole dataset back to dataset object

drug_dataset.reset_format()

Creating a validation set. We won't use the Test dataset for this, but instead create a new set as best practice

In [39]:
# We start by chopping the original train dataset by 0.8

drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)

# We rename the automatically named 'test' split to 'validation' by removing the "test" key from drug_dataset_clean and get its value. And then assign this value to a new key "validation" in drug_dataset_clean.

drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")

#add the original 'test' (which we had not touched before) set to the clean dataset dict.

#now our drug_dataset_clean should contain 3 separate keys: Train (which is 80% of the original Train), Test (20% of the original train) and Test (which is just the original test)

drug_dataset_clean["test"] = drug_dataset["test"]

drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Saving datasets in either Arrow / JSON / CSV / Parquet format

In [44]:
# Let's start by saving in Arrow format, directly to disk.

# Note that this saves to the Google Collab local ephemeral environment (i.e. in /content) which will be lost once the runtime ends

# to save it on google drive use : drug_dataset_clean.save_to_disk("/content/drive/My Drive/drug-reviews")


drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [45]:
# Once the dataset is saved we can load it again

from datasets import load_from_disk

drug_dataset_loaded = load_from_disk("drug-reviews")

drug_dataset_loaded

DatasetDict({
    train: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [46]:
# For the JSON / CSV formats, we have to save each split in the data (i.e. Test, Train, Val) separately.

# Remember, drug_dataset_clean contains 3 keys (which is the splits) and the values (the separate datasets). We iterate over these to save them separately with the right name

# We save them in a JSON Lines format by iterating over each key (here "split") and assigning the right value to each split.

for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

In [54]:
# Let's load the data set up again, again it has to be done individually

data_files = {
    "train" : "drug-reviews-train.jsonl",
    "validation" : "drug-reviews-validation.jsonl",
    "test" : "drug-reviews-test.jsonl",
}

drug_dataset_loaded = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]